In [1]:
import yfinance as yf
import pandas_ta
import talib
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [2]:
# Download currency quotes with library yfinance
df = yf.download('EURUSD=X', 
                      start='2022-02-20', 
                      end='2023-05-01',
                      interval='1h', 
                      progress=False)
df

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-02-21 00:00:00+00:00,1.131862,1.133787,1.131734,1.133401,1.133401,0
2022-02-21 01:00:00+00:00,1.133530,1.136364,1.133273,1.135718,1.135718,0
2022-02-21 02:00:00+00:00,1.135847,1.136493,1.135718,1.136105,1.136105,0
2022-02-21 03:00:00+00:00,1.136105,1.137010,1.136105,1.136364,1.136364,0
2022-02-21 04:00:00+00:00,1.136493,1.136880,1.136235,1.136493,1.136493,0
...,...,...,...,...,...,...
2023-04-28 18:00:00+01:00,1.103266,1.103509,1.102536,1.102657,1.102657,0
2023-04-28 19:00:00+01:00,1.102657,1.103022,1.101928,1.102050,1.102050,0
2023-04-28 20:00:00+01:00,1.102293,1.102657,1.101564,1.102414,1.102414,0


In [3]:
df.reset_index(inplace=True)
df['Datetime'] = [x.tz_convert(None) for x in df['Datetime']]

df.drop('Adj Close', axis=1, inplace=True)
df.drop('Volume', axis=1, inplace=True)

In [4]:
# Add technical indicators with library pandas_ta
df.ta.ema(close=df['Close'], length=21, offset=3, append=True)
df.ta.ema(close=df['Close'], length=8, offset=3, append=True)
df.ta.ema(close=df['Close'], length=50, offset=3, append=True)

df.ta.rsi(close=df['Close'], length=14, offset=3, append=True)
df.ta.stoch(df['High'], df['Low'], df['Close'], k=9, d=3, smooth_k=3, offset=3, append=True)
df.ta.stoch(df['High'], df['Low'], df['Close'], k=21, d=3, smooth_k=3, offset=3, append=True)

df.ta.bbands(length=20, std=2, mamode='sma', offset=3, append=True)

,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
7368,1.097759,1.101610,1.105461,0.699098,0.622572
7369,1.097684,1.101695,1.105707,0.728285,0.665304
7370,1.097680,1.101701,1.105723,0.730072,0.670179
7371,1.097685,1.101677,1.105669,0.724748,0.699098


In [5]:
df['Year'] = [x.year for x in df['Datetime']]
df['Month'] = [x.month for x in df['Datetime']]
df['Day'] = [x.day for x in df['Datetime']]
df['Day of week'] = [x.dayofweek for x in df['Datetime']]
df['Hour'] = [x.hour for x in df['Datetime']]

df.drop('Datetime', axis=1, inplace=True)
df

,Open,High,Low,Close,EMA_21,EMA_8,EMA_50,RSI_14,STOCHk_9_3_3,STOCHd_9_3_3,...,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,Year,Month,Day,Day of week,Hour
0,1.131862,1.133787,1.131734,1.133401,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022,2,21,0,0
1,1.133530,1.136364,1.133273,1.135718,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022,2,21,0,1
2,1.135847,1.136493,1.135718,1.136105,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022,2,21,0,2
3,1.136105,1.137010,1.136105,1.136364,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022,2,21,0,3
4,1.136493,1.136880,1.136235,1.136493,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022,2,21,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7368,1.103266,1.103509,1.102536,1.102657,1.101369,1.100776,1.102105,59.773085,58.525296,32.159028,...,1.097759,1.101610,1.105461,0.699098,0.622572,2023,4,28,4,17
7369,1.102657,1.103022,1.101928,1.102050,1.101641,1.101573,1.102193,59.773085,80.671567,54.796275,...,1.097684,1.101695,1.105707,0.728285,0.665304,2023,4,28,4,18
7370,1.102293,1.102657,1.101564,1.102414,1.101800,1.101976,1.102240,55.583026,91.869119,77.021994,...,1.097680,1.101701,1.105723,0.730072,0.670179,2023,4,28,4,19
7371,1.102171,1.102536,1.101807,1.101928,1.101878,1.102128,1.102256,52.609444,82.730199,85.090295,...,1.097685,1.101677,1.105669,0.724748,0.699098,2023,4,28,4,20


In [6]:
# Connect to MongoDb, load the collection that was create after parsing
client = MongoClient('localhost:27017')
db = client['data']
news = db['news']

In [9]:
new_df = pd.DataFrame(list(news.find({})))
new_df = new_df[['dtime', 'currency', 'country', 'importance', 'event', 'prev', 'forecast', 'actual']]
new_df

,dtime,currency,country,importance,event,prev,forecast,actual
0,2022-02-21 00:00:00,USD,United_States,Holiday,United States - Presidents' Day,-,-,-
1,2022-02-21 05:30:00,USD,United_States,2,Investing.com Gold Index,56.0%,,56.8%
2,2022-02-21 05:30:00,USD,United_States,2,Investing.com S&P 500 Index,70.8%,,70.4%
3,2022-02-21 05:30:00,EUR,Europe,2,Investing.com EUR/USD Index,54.8%,,57.6%
4,2022-02-21 07:00:00,EUR,Germany,1,German PPI (YoY) (Jan),24.2%,24.2%,25.0%
...,...,...,...,...,...,...,...,...
8637,2023-02-03 15:00:00,USD,United_States,1,ISM Non-Manufacturing New Orders (Jan),45.2,,60.4
8638,2023-02-03 15:00:00,USD,United_States,3,ISM Non-Manufacturing PMI (Jan),49.2,50.4,55.2
8639,2023-02-03 15:00:00,USD,United_States,1,ISM Non-Manufacturing Prices (Jan),68.1,,67.8
8640,2023-02-03 18:00:00,USD,United_States,2,U.S. Baker Hughes Oil Rig Count,609,,599


In [10]:
# Split datetime
new_df['Year'] = [x.year for x in new_df['dtime']]
new_df['Month'] = [x.month for x in new_df['dtime']]
new_df['Day'] = [x.day for x in new_df['dtime']]
new_df['Day of week'] = [x.dayofweek for x in new_df['dtime']]
new_df['Hour'] = [x.hour for x in new_df['dtime']]

In [11]:
# Replace Holiday value
new_df['importance'].replace({'Holiday': 0}, inplace=True)
new_df['importance'] = new_df['importance'].astype('int8')

In [12]:
# CLean data of columns prev, forecast, actual
for i in ['prev', 'forecast', 'actual']:
    new_df[i].replace(['', '-'], 0, inplace=True)
    new_df[i].replace(['%', 'K', 'B', 'M', 'T', ','], '', inplace=True, regex=True)
    new_df[i]= new_df[i].astype('float')
    
# Create columns of the difference between the previous, predicted and actual values (as a ratio) according to importance
new_df['prev_change_actual'] = new_df.apply(lambda x: x['importance'] * (x['actual'] - x['prev'])/(x['prev'] + 0.000000001), axis=1)
new_df['fore_change_actual'] = new_df.apply(lambda x: x['importance'] * (x['actual'] - x['forecast'])/(x['forecast'] + 0.000000001), axis=1)
new_df['prev_change_fore'] = new_df.apply(lambda x: x['importance'] * (x['forecast'] - x['prev'])/(x['prev'] + 0.000000001), axis=1)

In [13]:
# Grouping data by date, time and currency and aggregate other columns
new_df = new_df.groupby(['Year', 'Month', 'Day', 'Day of week', 'Hour', 'currency'])\
    .agg({'prev_change_actual': ['mean', 'std'], 'fore_change_actual': ['mean', 'std'], 
          'prev_change_fore': ['mean', 'std'], 
          'country': [lambda x: ' '.join(x)], 'event': [lambda x: ' '.join(x)]})\
    .reset_index()
new_df.head()

Year Month Day Day of week Hour currency prev_change_actual            \
                                                          mean       std   
0  2022     2  21           0    0      USD           0.000000       NaN   
1  2022     2  21           0    5      EUR           0.102190       NaN   
2  2022     2  21           0    5      USD           0.008636  0.028193   
3  2022     2  21           0    7      EUR          -0.543471  0.815335   
4  2022     2  21           0    8      EUR           0.082271  0.089909   

  fore_change_actual               prev_change_fore                \
                mean           std             mean           std   
0       0.000000e+00           NaN         0.000000           NaN   
1       1.152000e+11           NaN        -2.000000           NaN   
2       1.272000e+11  1.923330e+10        -2.000000  5.278886e-12   
3       4.831956e-01  6.365909e-01        -0.700000  9.899495e-01   
4       7.292040e-02  7.529295e-02         0.008553  1.565721e-02   

                                        country  \
                                       <lambda>   
0                                 United_States   
1                                        Europe   
2                   United_States United_States   
3                               Germany Germany   
4  France France France Germany Germany Germany   

                                               event  
                                            <lambda>  
0                    United States - Presidents' Day  
1                        Investing.com EUR/USD Index  
2  Investing.com Gold Index Investing.com S&P 500...  
3    German PPI (YoY)  (Jan) German PPI (MoM)  (Jan)  
4  French Manufacturing PMI  (Feb) French S&P Glo...

In [14]:
# Delete 2-level columns and rename
new_df = new_df.T.reset_index()

new_df['level_0'] = new_df.apply(lambda x: x['level_0'] + '_' + x['level_1'] if x['level_1'] in ['mean', 'std'] 
                       else x['level_0'], axis=1)
new_df.drop('level_1', axis=1, inplace=True)

new_df = new_df.T.rename(columns=new_df.T.iloc[0]).loc[1:]

# Change types of columns
contin = ['prev_change_actual_mean', 'prev_change_actual_std', 'fore_change_actual_mean', 'fore_change_actual_std',
       'prev_change_fore_mean', 'prev_change_fore_std']

for i in contin:
    new_df[i] = new_df[i].astype('float')

for i in ['Year', 'Month', 'Day', 'Day of week', 'Hour']:
    new_df[i] = new_df[i].astype('int')

new_df.head()

,Year,Month,Day,Day of week,Hour,currency,prev_change_actual_mean,prev_change_actual_std,fore_change_actual_mean,fore_change_actual_std,prev_change_fore_mean,prev_change_fore_std,country,event
1,2022,2,21,0,5,EUR,0.102190,NaN,1.152000e+11,NaN,-2.000000,NaN,Europe,Investing.com EUR/USD Index
2,2022,2,21,0,5,USD,0.008636,0.028193,1.272000e+11,1.923330e+10,-2.000000,5.278886e-12,United_States United_States,Investing.com Gold Index Investing.com S&P 500...
3,2022,2,21,0,7,EUR,-0.543471,0.815335,4.831956e-01,6.365909e-01,-0.700000,9.899495e-01,Germany Germany,German PPI (YoY) (Jan) German PPI (MoM) (Jan)
4,2022,2,21,0,8,EUR,0.082271,0.089909,7.292040e-02,7.529295e-02,0.008553,1.565721e-02,France France France Germany Germany Germany,French Manufacturing PMI (Feb) French S&P Glo...
5,2022,2,21,0,9,EUR,-0.003424,0.227322,-4.267500e+08,8.535000e+08,-0.237370,5.086247e-01,Greece Europe Europe Europe,Greek Current Account (YoY) Manufacturing PMI ...


In [15]:
# The price of a currency pair directly correlates with the first currency and inversely correlates with the second currency.
# Therefore, the factors of the second currency are multiplied by -1.
for i in contin:
    new_df[i] = new_df.apply(lambda x: x[i] * -1 if x['currency'] == 'USD' else x[i], axis=1)

# Grouping data by date and time
new_df.drop('currency', axis=1, inplace=True)
new_df = new_df.groupby(['Year', 'Month', 'Day', 'Day of week', 'Hour']).agg({
        'prev_change_actual_mean': 'sum', 'prev_change_actual_std': 'sum', 'fore_change_actual_mean': 'sum', 
        'fore_change_actual_std': 'sum', 'prev_change_fore_mean': 'sum', 'prev_change_fore_std': 'sum',
        'country': lambda x: ' '.join(x), 'event': lambda x: ' '.join(x)
}).reset_index()

new_df.head()

,Year,Month,Day,Day of week,Hour,prev_change_actual_mean,prev_change_actual_std,fore_change_actual_mean,fore_change_actual_std,prev_change_fore_mean,prev_change_fore_std,country,event
0,2022,2,21,0,5,0.093554,-0.028193,-1.200000e+10,-1.923330e+10,4.514611e-12,-5.278886e-12,Europe United_States United_States,Investing.com EUR/USD Index Investing.com Gold...
1,2022,2,21,0,7,-0.543471,0.815335,4.831956e-01,6.365909e-01,-7.000000e-01,9.899495e-01,Germany Germany,German PPI (YoY) (Jan) German PPI (MoM) (Jan)
2,2022,2,21,0,8,0.082271,0.089909,7.292040e-02,7.529295e-02,8.552896e-03,1.565721e-02,France France France Germany Germany Germany,French Manufacturing PMI (Feb) French S&P Glo...
3,2022,2,21,0,9,-0.003424,0.227322,-4.267500e+08,8.535000e+08,-2.373696e-01,5.086247e-01,Greece Europe Europe Europe,Greek Current Account (YoY) Manufacturing PMI ...
4,2022,2,21,0,10,-0.094203,0.000000,-6.250000e+08,0.000000e+00,-1.000000e+00,0.000000e+00,Germany,German 12-Month Bubill Auction


In [16]:
df = df.merge(new_df, how='left', on=['Year', 'Month', 'Day', 'Day of week', 'Hour'])
df

,Open,High,Low,Close,EMA_21,EMA_8,EMA_50,RSI_14,STOCHk_9_3_3,STOCHd_9_3_3,...,Day of week,Hour,prev_change_actual_mean,prev_change_actual_std,fore_change_actual_mean,fore_change_actual_std,prev_change_fore_mean,prev_change_fore_std,country,event
0,1.131862,1.133787,1.131734,1.133401,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.133530,1.136364,1.133273,1.135718,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.135847,1.136493,1.135718,1.136105,NaN,NaN,NaN,NaN,NaN,NaN,...,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.136105,1.137010,1.136105,1.136364,NaN,NaN,NaN,NaN,NaN,NaN,...,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.136493,1.136880,1.136235,1.136493,NaN,NaN,NaN,NaN,NaN,NaN,...,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7368,1.103266,1.103509,1.102536,1.102657,1.101369,1.100776,1.102105,59.773085,58.525296,32.159028,...,4,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7369,1.102657,1.103022,1.101928,1.102050,1.101641,1.101573,1.102193,59.773085,80.671567,54.796275,...,4,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7370,1.102293,1.102657,1.101564,1.102414,1.101800,1.101976,1.102240,55.583026,91.869119,77.021994,...,4,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7371,1.102171,1.102536,1.101807,1.101928,1.101878,1.102128,1.102256,52.609444,82.730199,85.090295,...,4,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Recognize price action pattern
candle_names = talib.get_function_groups()['Pattern Recognition']

# create columns for each pattern
for candle in candle_names:
    df[candle] = getattr(talib, candle)(df['Open'], df['High'], df['Low'], df['Close'])

df = df.loc[:, (df != 0).any(axis=0)]
df

,Unnamed: 0,Open,High,Low,Close,EMA_21,EMA_8,EMA_50,RSI_14,STOCHk_9_3_3,...,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLXSIDEGAP3METHODS
0,0,1.131862,1.133787,1.131734,1.133401,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1,1.133530,1.136364,1.133273,1.135718,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2,1.135847,1.136493,1.135718,1.136105,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,3,1.136105,1.137010,1.136105,1.136364,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,1.136493,1.136880,1.136235,1.136493,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7368,7368,1.103266,1.103509,1.102536,1.102657,1.101369,1.100776,1.102105,59.773085,58.525296,...,-100,0,0,0,0,0,0,0,0,0
7369,7369,1.102657,1.103022,1.101928,1.102050,1.101641,1.101573,1.102193,59.773085,80.671567,...,-100,0,0,0,0,0,0,0,0,0
7370,7370,1.102293,1.102657,1.101564,1.102414,1.101800,1.101976,1.102240,55.583026,91.869119,...,0,100,0,0,0,0,0,0,0,0
7371,7371,1.102171,1.102536,1.101807,1.101928,1.101878,1.102128,1.102256,52.609444,82.730199,...,-100,0,0,0,0,0,0,0,0,0
